In [1]:
# Load testdatabase into memory, ignore duplicate rows
# Import more data for better results
import sqlite3
import numpy as np
import math
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('testdatabase.sqlite3')
c = conn.cursor()
c.execute('SELECT DISTINCT standard_inchi,protein_sequence,standard_value FROM small_interactions')
d = c.fetchall()
conn.close()

In [2]:
# Convert inchi and protein sequence into ASCII, convert IC50 into binary True/False 
din=[[0] for x in range(len(d))]
dpr=[[0] for x in range(len(d))]
dca=[[0] for x in range(len(d))]
i=0
for row in d:
    din[i] = [ord(c) for c in d[i][0]] # inchis
    dpr[i] = [ord(c) for c in d[i][1]] # proteins
    dca[i] = 0 if d[i][2]>800 else 1   # categories - can adjust this threshold for better results
    i += 1
count1 = 0   
count0 = 0
for i in dca:
    if i == 1:
        count1+=1
    else:
        count0+=1
print (count0)
print (count1)

39040
20948


In [3]:
# Pad with zeros to max length of Inchis and Proteins
# Since we concatenate the two later, need max length overall to pad so every conc row has same length
maxlen=0
for row in range(len(din)):
    maxlen = len(din[row]) if len(din[row])>maxlen else maxlen
    maxlen = len(dpr[row]) if len(dpr[row])>maxlen else maxlen # delete this if not conc later
for row in range(len(din)):
    din[row].extend([0 for x in range(maxlen-len(din[row]))])  
    dpr[row].extend([0 for x in range(maxlen-len(dpr[row]))])  # delete this if not conc later
#maxlen=0
#for row in range(len(dpr)):
#    maxlen = len(dpr[row]) if len(dpr[row])>maxlen else maxlen
#for row in range(len(dpr)):
#    dpr[row].extend([0 for x in range(maxlen-len(dpr[row]))])

nin = np.array(din).astype(float) # turn into Numpy Array for Keras - INICHIS (samples x asciis)
npr = np.array(dpr).astype(float) # turn into Numpy Array for Keras - PROTEINS (samples x asciis)
nca = np.array(dca).astype(float) # turn into Numpy Array for Keras - IC50s (samples x 0/1)

In [4]:
# Scale ASCII int to float(0..1) 
maxval = 0
for row in range(len(nin)):
    maxval = max(nin[row]) if max(nin[row])>maxval else maxval
for row in range(len(nin)):
    nin[row] = np.divide(nin[row],float(maxval))
maxval = 0
for row in range(len(npr)):
    maxval = max(npr[row]) if max(npr[row])>maxval else maxval
for row in range(len(npr)):
    npr[row] = np.divide(npr[row],float(maxval))

In [5]:
# Import Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils

Using TensorFlow backend.


In [6]:
# Create Train and Test inputs for Keras
# Concatenate Inchis and Proteins as a first try
X = np.concatenate((nin,npr),axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, nca, test_size=0.2, random_state=61)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
# one hot encode outputs - needed for Keras optimizer
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
# Build CNN with 
# 2x (1D Conv layer + MaxPooling)
# 2x Dense
# Can adjust:
# number of Conv1D layers
# number of Conv1D features (default=32, 16), size of Kernel (default=8,4)
# size of MaxPooling1D (default=2,2)
# number and size of Dense layers

model = Sequential()
model.add(Conv1D(1, (2), input_shape=(X_train.shape[1],1), activation='relu'))
model.add(MaxPooling1D(pool_size=(2)))
#model.add(Conv1D(16, (4), activation='relu'))
#model.add(MaxPooling1D(pool_size=(2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(2, activation='relu'))
#model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
batch_size = 128
epochs = 3
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 47990 samples, validate on 11998 samples
Epoch 1/3
47990/47990 [==============================] - 4s 75us/step - loss: 0.6506 - acc: 0.6728 - val_loss: 0.6098 - val_acc: 0.7055
Epoch 2/3
47990/47990 [==============================] - 3s 70us/step - loss: 0.5988 - acc: 0.7076 - val_loss: 0.5856 - val_acc: 0.7152
Epoch 3/3
47990/47990 [==============================] - 3s 70us/step - loss: 0.5855 - acc: 0.7119 - val_loss: 0.5777 - val_acc: 0.7216


In [9]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 597, 1)            3         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 298, 1)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 298, 1)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 298)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 598       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 607
Trainable params: 607
Non-trainable params: 0
_________________________________________________________________
None
